### MaskRCNN tensorflow to onnx conversion and TensorRT optimization. Balloon dataset

In [1]:
import os
os.chdir('..')

In [2]:
import tensorflow as tf
import onnx
import subprocess
from samples.balloon import balloon
from model import mask_rcnn_functional
from common.utils import tf_limit_gpu_memory
from common import inference_utils
from common.inference_optimize import maskrcnn_to_onnx, modify_onnx_model
tf_limit_gpu_memory(tf, 1000)

1 Physical GPUs, 1 Logical GPUs Memory limit: 1000
Physical GPU-devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
from common.config import CONFIG

In [4]:
%load_ext watermark
%watermark
%watermark --iversions

Last updated: 2021-06-24T02:10:04.096601+03:00

Python implementation: CPython
Python version       : 3.7.7
IPython version      : 7.16.1

Compiler    : GCC 7.3.0
OS          : Linux
Release     : 5.4.0-65-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 12
Architecture: 64bit

tensorflow: 2.2.0
onnx      : 1.8.1



In [5]:
CONFIG.update(balloon.BALLON_CONFIG)
CONFIG

{'image_shape': (512, 512, 3),
 'img_size': 512,
 'backbone': 'mobilenet',
 'meta_shape': 14,
 'num_classes': 2,
 'class_dict': {'balloon': 1, 'background': 0},
 'normalization': None,
 'image_min_dim': 300,
 'image_min_scale': 0,
 'image_max_dim': 512,
 'image_resize_mode': 'square',
 'use_mini_masks': False,
 'mini_mask_shape': (32, 32),
 'mask_shape': (28, 28),
 'epochs': 100,
 'gpu_num': 1,
 'batch_size': 1,
 'images_per_gpu': 1,
 'training': True,
 'log_per_steps': 5,
 'use_multiprocessing': True,
 'workers': 6,
 'callback': {'log_dir': 'logs/scalars',
  'reduce_lr_on_plateau': 0.98,
  'reduce_lr_on_plateau_patience': 10,
  'save_weights_only': True,
  'save_best_only': True,
  'histogram_freq': 0,
  'profile_batch': '1,2'},
 'backbone_strides': [4, 8, 16, 32, 64],
 'top_down_pyramid_size': 256,
 'rpn_anchor_scales': (32, 64, 128, 256, 512),
 'rpn_anchor_ratios': [0.5, 1, 2],
 'rpn_anchor_stride': 1,
 'rpn_train_anchors_per_image': 256,
 'max_gt_instances': 100,
 'rpn_bbox_std_dev

#### Prepare inference graph

In [6]:
checkpoint = [x for x in os.listdir(f'../tests/samples/balloon') 
              if 'maskrcnn_%s' % CONFIG['backbone'] in x]
checkpoint = checkpoint[0].split('.ckpt')[0] +'.ckpt'
checkpoint

'maskrcnn_mobilenet_ed3e7dd4c2e064d9dd92df2088834243_cp-0029.ckpt'

In [7]:
model_name = f"""maskrcnn_{CONFIG['backbone']}_{'_'.join(list(map(str, CONFIG['image_shape'])))}""" 
weights_path = os.path.join('..', 'tests', 'samples', 'balloon', checkpoint)
print(f'Weights path:\n{weights_path}\n\nModel name:\n{model_name}')

Weights path:
../tests/samples/balloon/maskrcnn_mobilenet_ed3e7dd4c2e064d9dd92df2088834243_cp-0029.ckpt

Model name:
maskrcnn_mobilenet_512_512_3


In [8]:
# Loading inference graph and import weights
inference_config = CONFIG
inference_config.update({'training': False})
inference_model = mask_rcnn_functional(config=inference_config)
inference_model = inference_utils.load_mrcnn_weights(model=inference_model,
                                                     weights_path=weights_path,
                                                     verbose=True
                                                    )

[MaskRCNN] Inference mode

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[MaskRCNN] Backbone architecture: mobilenet


/home/alexander/anaconda3/envs/tf_env/lib/python3.7/site-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '



Weights for inference graph will be transferred from training graph

[MaskRCNN] Training mode

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[MaskRCNN] Backbone architecture: mobilenet
MaskRCNN Losses:
rpn_class_loss: <layers.losses.RPNClassLoss object at 0x7f6cfc5b4b10>
rpn_bbox_loss: <layers.losses.RPNBboxLoss object at 0x7f6cfc2d7950>
mrcnn_class_loss: <layers.losses.MRCNNClassLoss object at 0x7f6cfc379bd0>
mrcnn_bbox_loss: <layers.losses.MRCNNBboxLoss object at 0x7f6ce87c2b50>
mrcnn_mask_loss: <layers.losses.MRCNNMaskLoss object at 0x7f6ce87c2c

Total params: 24,073,932
Trainable params: 23,755,980
Non-trainable params: 317,952
__________________________________________________________________________________________________
None


#### Convert model to .onnx with tf2onnx

In [9]:
input_spec = (
    tf.TensorSpec((CONFIG['batch_size'], *CONFIG['image_shape']), tf.float32, name="input_image"),
    tf.TensorSpec((CONFIG['batch_size'], CONFIG['meta_shape']), tf.float32, name="input_image_meta")
)
maskrcnn_to_onnx(model=inference_model, 
                 model_name = model_name,
                 input_spec=input_spec,
                 kwargs={'opset': 11}
                )

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


Cannot infer shape for mask_rcnn_inference/roi_align_classifier/Squeeze_1: mask_rcnn_inference/roi_align_classifier/Squeeze_1:0
Cannot infer shape for mask_rcnn_inference/roi_align_classifier/Unique: mask_rcnn_inference/roi_align_classifier/Unique:1


Successfully converted from tensorflow to .onnx: ../weights/maskrcnn_mobilenet_512_512_3.onnx


#### Load onnx model and check it 

In [10]:
# Load the ONNX model
model = onnx.load(f'../weights/{model_name}.onnx')
# Check that the IR is well formed
onnx.checker.check_model(model)
# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

graph tf2onnx (
  %input_image[FLOAT, 1x512x512x3]
  %input_image_meta[FLOAT, 1x14]
) initializers (
  %slice_axes__590[INT32, 1]
  %slice_axes__1150[INT32, 2]
  %new_shape__1563[INT64, 4]
  %mask_rcnn_inference/rpn_model_4/rpn_conv_shared/Conv2D/ReadVariableOp:0[FLOAT, 512x256x3x3]
  %mask_rcnn_inference/rpn_model_3/rpn_bbox_pred/Conv2D/ReadVariableOp:0[FLOAT, 12x512x1x1]
  %mask_rcnn_inference/rpn_model/rpn_class_raw/Conv2D/ReadVariableOp:0[FLOAT, 6x512x1x1]
  %mask_rcnn_inference/roi_align_mask/range/delta:0[INT32, scalar]
  %mask_rcnn_inference/roi_align_classifier/truediv/x:0[FLOAT, scalar]
  %mask_rcnn_inference/roi_align_classifier/range/start:0[INT32, scalar]
  %mask_rcnn_inference/roi_align_classifier/mul_2/y:0[INT32, scalar]
  %mask_rcnn_inference/roi_align_classifier/add/x:0[INT32, scalar]
  %mask_rcnn_inference/roi_align_classifier/PadV2/constant_values:0[INT32, scalar]
  %mask_rcnn_inference/roi/sub_4/x:0[INT32, scalar]
  %mask_rcnn_inference/roi/strided_slice_6:0[FLOAT, 6

#### Configure model for TensorRT

In [11]:
modify_onnx_model(model_path=f'../weights/{model_name}.onnx',
                  config=CONFIG,
                  verbose=True
                 )


Initial graph inputs: [Variable (input_image): (shape=[1, 512, 512, 3], dtype=float32), Variable (input_image_meta): (shape=[1, 14], dtype=float32)]

Initial graph outputs: [Variable (mrcnn_detection): (shape=[1, 100, 6], dtype=float32), Variable (fpnclf_mrcnn_class): (shape=['unk__2801', 1000, 2], dtype=float32), Variable (fpnclf_mrcnn_bbox_reshape): (shape=['unk__2802', 1000, 2, 4], dtype=float32), Variable (mrcnn_mask): (shape=[1, 100, 28, 28, 2], dtype=float32), Variable (roi): (shape=[1, 'unk__2803', 'unk__2804'], dtype=float32), Variable (concat_rpn_class): (shape=[1, 65472, 2], dtype=float32), Variable (concat_rpn_bbox): (shape=[1, 65472, 4], dtype=float32)]
Already cleared: mask_rcnn_inference/mrcnn_detection/Unique:1
Already cleared: mask_rcnn_inference/mrcnn_detection/Unique:2
Already cleared: mask_rcnn_inference/mrcnn_detection/TopKV2:0
Already cleared: mask_rcnn_inference/roi/top_anchors:0
Already cleared: mask_rcnn_inference/roi_align_classifier/Unique:1
Already cleared: 

#### TensorRT optimization

__With trtexec:__ 

In [12]:
os.environ['TRTEXEC']

'/home/alexander/TensorRT-7.2.3.4/bin/trtexec'

In [13]:
%%time

os.chdir('../weights')

# Construct appropriate command
fp16_mode = False
command = [os.environ['TRTEXEC'],
           f'--onnx={model_name}_trt_mod.onnx',
           f'--saveEngine={model_name}_trt_mod_fp32.engine',
            '--workspace=2048',
            '--explicitBatch',
            '--verbose',
          ]

# fp16 param
if fp16_mode:
    command[2].replace('32', '16')
    command.append('--fp16')

# tacticSources param
# Do not neeed on jetson with aarch64 architecture for now.
arch = os.uname().machine
if arch == 'x86_64':
    command.append('--tacticSources=-cublasLt,+cublas')
    
print(f'\nArch: {arch}\ntrtexec command list: {command}')

result = subprocess.run(command, capture_output=True, check=True)
# Print stdout inference result
print(result.stdout.decode('utf8')[-2495:])


Arch: x86_64
trtexec command list: ['/home/alexander/TensorRT-7.2.3.4/bin/trtexec', '--onnx=maskrcnn_mobilenet_512_512_3_trt_mod.onnx', '--saveEngine=maskrcnn_mobilenet_512_512_3_trt_mod_fp32.engine', '--workspace=2048', '--explicitBatch', '--verbose', '--tacticSources=-cublasLt,+cublas']
016
[06/24/2021-02:14:17] [V] [TRT] Allocated activation device memory of size 329411584
[06/24/2021-02:14:17] [V] [TRT] Assigning persistent memory blocks for various profiles
[06/24/2021-02:14:17] [I] Starting inference
[06/24/2021-02:14:20] [I] Warmup completed 0 queries over 200 ms
[06/24/2021-02:14:20] [I] Timing trace has 0 queries over 3.13159 s
[06/24/2021-02:14:20] [I] Trace averages of 10 runs:
[06/24/2021-02:14:20] [I] Average on 10 runs - GPU latency: 51.1195 ms - Host latency: 51.5417 ms (end to end 103.045 ms, enqueue 2.57648 ms)
[06/24/2021-02:14:20] [I] Average on 10 runs - GPU latency: 51.1507 ms - Host latency: 51.5731 ms (end to end 102.158 ms, enqueue 2.75852 ms)
[06/24/2021-02:14

__With python TensorRT API:__


In [14]:
import tensorrt as trt

In [15]:
max_batch_size = 1
# Precision mode
fp16_mode = False
# Workspace size in Mb
wspace_size = 2048

In [16]:
%%time

# Init TensorRT Logger
TRT_LOGGER = trt.Logger(trt.Logger.VERBOSE)
# Init TensorRT plugins
trt.init_libnvinfer_plugins(TRT_LOGGER, "")
# Set tensorrt-prepared onnx model
onnx_model_path = f'../weights/{model_name}_trt_mod.onnx'
# Use explicit batch
explicit_batch = 1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)

with trt.Builder(TRT_LOGGER) as builder, \
        builder.create_builder_config() as builder_config, \
        builder.create_network(explicit_batch) as network, \
        trt.OnnxParser(network, TRT_LOGGER) as parser:

    with open(onnx_model_path, 'rb') as model:
        parser.parse(model.read())

    print('Num of detected layers: ', network.num_layers)
    print('Detected inputs: ', network.num_inputs)
    print('Detected outputs: ', network.num_outputs)
    
    # Workspace size
    # 1e6 bytes == 1Mb
    builder_config.max_workspace_size = int(1e6 * wspace_size)
    
    # Precision mode
    if fp16_mode:
        builder_config.set_flag(trt.BuilderFlag.FP16)
    
    # Max batch size
    builder.max_batch_size = max_batch_size
    
    # Set the list of tactic sources
    # Do not need for Jetson with aarch64 architecture for now
    arch = os.uname().machine
    if arch == 'x86_64':
        tactic_source = 1 << int(trt.TacticSource.CUBLAS) | 0 << int(trt.TacticSource.CUBLAS_LT)
        builder_config.set_tactic_sources(tactic_source)
        
    
    # Make TensorRT engine
    engine = builder.build_engine(network, builder_config)
    
    # Save TensorRT engine
    if fp16_mode:
        trt_model_name = f'../weights/{model_name}_fp16_trt.engine'
    else:
        trt_model_name = f'../weights/{model_name}_fp32_trt.engine'

    with open(trt_model_name, "wb") as f:
        f.write(engine.serialize())

Num of detected layers:  233
Detected inputs:  1
Detected outputs:  2
CPU times: user 17.8 s, sys: 3.05 s, total: 20.9 s
Wall time: 33.7 s
